In [1]:
import pyarrow as pa
import pyarrow.compute as pc

batch = pa.RecordBatch.from_arrays([range(1,1_000_000),
                                    range(1,1_000_000),
                                    range(1,1_000_000)],
                                   names=["x", "y", "z"])
print(batch.nbytes / 1024**2)

with pa.ipc.new_file("test.arrow", schema=batch.schema) as f:
    for i in range(50):
        f.write_batch(batch)

22.888160705566406


In [2]:
!ls -lh test.arrow

-rw-r--r-- 1 root root 1.2G Feb 13 15:35 test.arrow


In [7]:
import pyarrow as pa
import pyarrow.ipc

In [1]:
# can't read a 1 GB file given a 512 MB mem limit
# with pa.ipc.open_file("test.arrow") as f:
#     t = f.read_all()

In [2]:
import mmap

In [3]:
with open("test.arrow", "rb") as f:
    mm = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)

In [5]:
mm[:10]

b'ARROW1\x00\x00\xff\xff'

In [8]:
with pa.ipc.open_file(mm) as f:
    t = f.read_all()

In [10]:
import pyarrow.compute as pc

In [11]:
pc.sum(t["x"])

<pyarrow.Int64Scalar: 24999975000000>

In [12]:
pc.sum(t["y"])

<pyarrow.Int64Scalar: 24999975000000>

In [13]:
pc.sum(t["z"])

<pyarrow.Int64Scalar: 24999975000000>